In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from dataloader.rosen import RosenData
from visualize import get_vae, get_model, set_random, build_estimator
config = {
    'estimator': 'nngp',
    'random_seed': 43,
    'n_dim': 10,
    'data_size': 2000,
    'data_split': [0.2, 0.1, 0.1, 0.6],
    'update_size': 100,
    'al_iterations': 10,
    'verbose': True,
    'use_cache': True,
    'layers': [10, 128, 64, 32, 1],
    'patience': 5,
    'retrain': True,
    'model_path': 'model/data/rosen_visual.ckpt',
    'epochs': 20_000,
}


In [ ]:
rosen = RosenData(
    config['n_dim'], config['data_size'], config['data_split'],
    use_cache=config['use_cache'])

x_pool, y_pool = rosen.dataset('pool')
x_train, y_train = rosen.dataset('train')
x_val, y_val = rosen.dataset('train')

set_random(config['random_seed'])
x_full = np.concatenate((x_train, x_pool))
y_full = np.concatenate((y_train, y_pool))
model = get_model(config['retrain'], config['model_path'], (x_full, y_full), (x_val, y_val), epochs=30_000)

In [ ]:
estimator = build_estimator('mcdue', model)
estimations = estimator.estimate(x_val, x_train, y_train)
predictions = model(x_val).cpu().numpy()
error = 2*np.abs(predictions-y_val)/(predictions+y_val)
plt.figure(figsize=(12, 9))
plt.scatter(estimations/max(estimations), error)
plt.xlabel('Uncertainty')
plt.ylabel('Error')

